# MonteBERT
Downstream masked Language Modeling task using a Transformer, based on DistilBERT and trained from scratch.

## Import Libraries

In [6]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
import os

## Get Dataset

In [3]:
paths = [str(x) for x in Path(".").glob("**/*.txt")]
print(paths)

['data\\cristo.txt']


## Train a Tokenizer

In [4]:
tokenizer = ByteLevelBPETokenizer() #initialize tokenizer

In [5]:
#customise training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

## Save files to disk

In [11]:
token_dir = './content/MonteBERT'
if not os.path.exists(token_dir):
    os.makedirs(token_dir)
tokenizer.save_model(token_dir)

['./content/MonteBERT\\vocab.json', './content/MonteBERT\\merges.txt']